# Solution exercise a.

Copy the cell below and paste in [the main notebook](../modeling.ipynb).

In [ ]:
# Some arbitrary hyperparameters
min_child_weights = [1, 2, 4, 8, 10]

for weight in min_child_weights:
    hyperparams["min_child_weight"] = weight
    
    trial_name = f"algorithm-mode-trial-{create_date()}-weight-{weight}"
    trial = Trial.create(trial_name=trial_name, 
                     experiment_name=customer_churn_experiment.experiment_name,
                     sagemaker_boto_client=boto3.client('sagemaker'))
    
    xgb = sagemaker.estimator.Estimator(image_uri=docker_image_name,
                                        role=role,
                                        hyperparameters=hyperparams,
                                        instance_count=1, 
                                        instance_type='ml.m4.xlarge',
                                        output_path=f's3://{bucket}/{prefix}/output',
                                        base_job_name="demo-xgboost-customer-churn",
                                        sagemaker_session=sm_sess)
    
    # Look at the wait=False below
    print(f"Starting training job for trial {trial_name}!")
    xgb.fit(inputs={
                      'train': s3_input_train,
                      'validation': s3_input_validation
                    },
            experiment_config={
                "ExperimentName": customer_churn_experiment.experiment_name, 
                "TrialName": trial.trial_name,
                "TrialComponentDisplayName": "Training",
            },
            wait=False
           )